# Recuperer les donnees de NVDA

In [2]:
!pip install yfinance

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 61.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 65.5 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 61.9 MB/s  0:00:00
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15635 sha256=dec33928cd3dbf0c53e33ef242d29ae55b29b9080331b2602e1b90c6f87d1cea
  Stored in directory: /home/codespace/.cache/pip/wheels/cc/bd/6f/664d62c99327abeef7d86489e6631cbf45b56fbf7ef1d6ef00
Successfully built multitasking
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10/10 [yfinance]/10 [curl_cffi]]

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [5]:
!python3 -m pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 41.9 MB/s  0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 25.3
    Uninstalling pip-25.3:
      Successfully uninstalled pip-25.3


In [13]:
tickers = ["NVDA", "HG=F"]

for ticker in tickers :
    data = yf.download(ticker, start="2020-01-01", end="2026-02-19")
    print(data["Close"])


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Ticker            NVDA
Date                  
2020-01-02    5.971078
2020-01-03    5.875504
2020-01-06    5.900145
2020-01-07    5.971574
2020-01-08    5.982777
...                ...
2026-02-11  190.050003
2026-02-12  186.940002
2026-02-13  182.809998
2026-02-17  184.970001
2026-02-18  187.979996

[1540 rows x 1 columns]
Ticker        HG=F
Date              
2020-01-02  2.8330
2020-01-03  2.7985
2020-01-06  2.8005
2020-01-07  2.8040
2020-01-08  2.8190
...            ...
2026-02-11  5.9490
2026-02-12  5.7710
2026-02-13  5.7925
2026-02-17  5.6330
2026-02-18  5.7935

[1543 rows x 1 columns]
